In [14]:
# Imports
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Install the webdriver_manager pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install webdriver_manager

In [15]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/matt/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache


In [17]:
def start_kayak(city_from, city_to, date_start, date_end):
    """City codes - it's the IATA codes!
    Date format -  YYYY-MM-DD"""
    
    # Starting kayak chrome 
    kayak = ('https://www.kayak.com/flights/' + city_from + '-' + city_to +
             '/' + date_start + '-flexible/' + date_end + '-flexible?sort=bestflight_a')
    driver.get(kayak)

In [56]:
def page_scrape():
    xp_info = '//*[@id="Iojd-info-leg-0"]/div'
    info = driver.find_element(By.XPATH, xp_info)
    print(info.text.split("\n"))
    

In [19]:
start_kayak("LGW", "CDG", "2022-03-15", "2022-03-22")

In [57]:
page_scrape()

['3/16', 'Wed', '4:45 pm – 7:00 pm', 'easyJet', 'nonstop', '1h 15m', 'LGW', '‐', 'CDG']
